## test code part 1

In [1]:
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain_intro.chatbot import chat_model

In [2]:
messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about
        healthcare. Only answer healthcare-related questions."""
    ),
    HumanMessage(content="What is Medicaid managed care?"),
]

In [3]:
chat_model.invoke(messages)

AIMessage(content='Medicaid managed care is a system in which state Medicaid programs contract with private health insurance companies to provide healthcare services to Medicaid beneficiaries. In this model, the managed care organizations (MCOs) are responsible for coordinating and delivering a range of healthcare services, including preventive care, hospital services, and specialty care, to enrolled members.\n\nKey features of Medicaid managed care include:\n\n1. **Care Coordination**: MCOs help coordinate care among various providers to ensure that beneficiaries receive appropriate and timely services.\n\n2. **Cost Control**: By managing the care provided to beneficiaries, MCOs aim to control costs and improve efficiency in the delivery of healthcare services.\n\n3. **Access to Services**: Beneficiaries typically have a network of providers from which to choose, and MCOs may offer additional services beyond what traditional Medicaid covers.\n\n4. **Quality Improvement**: Many MCOs im

In [4]:
from langchain.prompts import ChatPromptTemplate

review_template_str = """Your job is to use patient
reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed
as possible, but don't make up any information that's not
from the context. If you don't know an answer, say you don't know.

{context}

{question}
"""

review_template = ChatPromptTemplate.from_template(review_template_str)

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_template.format(context=context, question=question)

"Human: Your job is to use patient\nreviews to answer questions about their experience at a hospital.\nUse the following context to answer questions. Be as detailed\nas possible, but don't make up any information that's not\nfrom the context. If you don't know an answer, say you don't know.\n\nI had a great stay!\n\nDid anyone have a positive experience?\n"

In [5]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_system_template_str = """Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_prompt_template.format_messages(context=context, question=question)

[SystemMessage(content="Your job is to use patient\nreviews to answer questions about their experience at a\nhospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n\nI had a great stay!\n"),
 HumanMessage(content='Did anyone have a positive experience?')]

## test code part 2

In [1]:
from langchain_intro.chatbot import review_chain

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_chain.invoke({"context": context, "question": question})

AIMessage(content='Yes, one patient mentioned that they had a great stay at the hospital, indicating a positive experience.', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 83, 'total_tokens': 103}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-9e440f5f-9648-454b-b1c7-1f4e99719b65-0')

## test code 3

In [9]:
import dotenv
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from elasticsearch import Elasticsearch

dotenv.load_dotenv()

# Configuration for Elasticsearch
ELASTICSEARCH_URL = "http://localhost:9200"
INDEX_NAME = "reviews_index"

# Defining the embedding model
embeddings = HuggingFaceEmbeddings()

# Set up ElasticsearchStore
reviews_vector_db = ElasticsearchStore(
    es_url=ELASTICSEARCH_URL,
    index_name=INDEX_NAME,
    embedding=embeddings,
    es_user="elastic",
    es_password="changeme",  # Replace with your password if security is enabled

)


# If you have documents to add, you can add them like this
# Assume `documents` is a list of Document objects
# documents = [Document(page_content="Some content", metadata={"source": "some source"})]
# reviews_vector_db.add_documents(documents)

# Question to search
question = """Has anyone complained about communication with the hospital staff?"""

# Perform similarity search
relevant_docs = reviews_vector_db.similarity_search(query=question, k=3)

# Display the relevant documents
print(relevant_docs[0].page_content)
print(relevant_docs[1].page_content)
print(relevant_docs[2].page_content)


I encountered some communication issues during my stay. The medical staff seemed disorganized, and it led to confusion about my treatment plan.
I encountered some issues with the nursing staff's communication. It seemed like there was a lack of coordination, leading to confusion about my medication schedule and treatment plan.
The hospital staff were friendly and attentive, making my stay more pleasant. However, there were occasional lapses in communication that caused confusion about my treatment plan.


## test code 4

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_elasticsearch import ElasticsearchStore
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import (
    create_openai_functions_agent,
    AgentExecutor,
)
from langchain.tools.base import StructuredTool
from langchain import hub
from langchain_intro.tools import get_current_wait_time
from pydantic import BaseModel
import dotenv
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

dotenv.load_dotenv(encoding='utf-8')

# Define the prompt templates for the Reviews tool
review_template_str = """Your job is to use patient
reviews to answer questions about their experience at
a hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],
        template=review_template_str,
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template="{question}",
    )
)

messages = [review_system_prompt, review_human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)

chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

# Elasticsearch connection details
ELASTICSEARCH_URL = "http://localhost:9200"
INDEX_NAME = "reviews_index"

# Initialize ElasticsearchStore
elastic_vector_search = ElasticsearchStore(
    es_url=ELASTICSEARCH_URL,
    index_name=INDEX_NAME,
    embedding= HuggingFaceEmbeddings(),
)



/usr/local/python/3.10.13/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Create a retriever for reviews
from operator import itemgetter
reviews_retriever = elastic_vector_search.as_retriever(k=10)

review_chain = (
    {
        "context": itemgetter("question") | reviews_retriever,
        "question": itemgetter("question")
    }
    #{"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | chat_model
    | StrOutputParser()
)



In [4]:
review_chain.input_schema.schema()

{'title': 'RunnableParallel<context,question>Input',
 'type': 'object',
 'properties': {'question': {'title': 'Question'}}}

In [5]:

class WaitTimeInput(BaseModel):
    hospital_name: str

waits_tool = StructuredTool.from_function(
    name="Waits",
    func=get_current_wait_time,
    description="""Use when asked about current wait times
    at a specific hospital. This tool can only get the current
    wait time at a hospital and does not have any information about
    aggregate or historical wait times. This tool returns wait times in
    minutes. Do not pass the word "hospital" as input,
    only the hospital name itself. For instance, if the question is
    "What is the wait time at hospital A?", the input should be "A".
    """,
    args_schema=WaitTimeInput,
)

review_tool = review_chain.as_tool(
    name="Reviews",
    description="""Handles structured inputs for review processing.
    Useful when you need to answer questions about patient reviews or experiences at the hospital.
    Not useful for answering questions about specific visit details such as payer, billing, treatment, diagnosis,
    chief complaint, hospital, or physician information. Pass the entire question as input to the tool.
    For instance, if the question is "What do patients think about the triage system?", the input should be
    "What do patients think about the triage system?".
    """,
)

# Update tools list with the corrected `Waits` tool
chat_tools = [review_tool, waits_tool]

# Set up the agent using the defined tools
hospital_agent_prompt = hub.pull("hwchase17/openai-functions-agent")

agent_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

hospital_agent = create_openai_functions_agent(
    llm=agent_chat_model,
    prompt=hospital_agent_prompt,
    tools=chat_tools,
)

# Create the agent executor
hospital_agent_executor = AgentExecutor(
    agent=hospital_agent,
    tools=chat_tools,
    return_intermediate_steps=True,
    verbose=True,
)


/usr/local/python/3.10.13/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(
/usr/local/python/3.10.13/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [6]:
hospital_agent_executor.invoke(
    {"input": "What have patients said about their comfort at the hospital?"}
)



> Entering new AgentExecutor chain...

Invoking: `Reviews` with `{'question': 'What have patients said about their comfort at the hospital?'}`


Patients have expressed mixed feelings about their comfort at the hospital. 

1. Douglas Crawford, who visited Rose Inc, mentioned that while the medical care was adequate, the lack of privacy in shared rooms made it challenging for him to rest and recover.

2. Nancy Collins, a patient at Richardson-Powell, noted that although the doctors were knowledgeable and the hospital environment was clean, the outdated and uncomfortable beds negatively impacted her overall comfort.

3. Justin Robertson, who stayed at Vaughn PLC, reported that while the medical care was exceptional, the higher-than-expected noise level in the ward affected his ability to rest.

4. Miranda Frey, a patient at Brown-Golden, appreciated the dedication of the hospital staff to patient care but found the uncomfortable beds made it difficult for her to get a good night's slee

{'input': 'What have patients said about their comfort at the hospital?',
 'output': "Patients have shared mixed experiences regarding their comfort at the hospital:\n\n1. **Douglas Crawford** (Rose Inc) mentioned that while the medical care was adequate, the lack of privacy in shared rooms made it difficult for him to rest and recover.\n\n2. **Nancy Collins** (Richardson-Powell) noted that although the doctors were knowledgeable and the hospital environment was clean, the outdated and uncomfortable beds negatively impacted her overall comfort.\n\n3. **Justin Robertson** (Vaughn PLC) reported that while the medical care was exceptional, the higher-than-expected noise level in the ward affected his ability to rest.\n\n4. **Miranda Frey** (Brown-Golden) appreciated the dedication of the hospital staff to patient care but found the uncomfortable beds made it hard for her to get a good night's sleep during her stay.\n\nOverall, patients have highlighted issues with bed comfort and noise le

In [7]:
hospital_agent_executor.invoke(
    {"input": "What is the current wait time at hospital C?"}
)



> Entering new AgentExecutor chain...

Invoking: `Waits` with `{'hospital_name': 'C'}`


4596The current wait time at hospital C is 4596 minutes.

> Finished chain.


{'input': 'What is the current wait time at hospital C?',
 'output': 'The current wait time at hospital C is 4596 minutes.',
 'intermediate_steps': [(AgentActionMessageLog(tool='Waits', tool_input={'hospital_name': 'C'}, log="\nInvoking: `Waits` with `{'hospital_name': 'C'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"hospital_name":"C"}', 'name': 'Waits'}}, response_metadata={'finish_reason': 'function_call'}, id='run-730eaa4e-ce5f-4f5c-8af5-fd0e1b9f1928')]),
   4596)]}